In [6]:
#Imports
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd


In [7]:
#Set path for splinter (chromedriver)
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
#url to scrape
url = 'https://mars.nasa.gov/news/'
#set url in splinter/chromedriver
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')


In [31]:
#Scrape the NASA Mars News Site and collect the latest News Title and Paragragh Text
news_title = soup.find(class_='content_title')
news_p = soup.find_all(class_='article_teaser_body')
print(news_title)
print(news_p)

<div class="content_title"><a href="/news/8326/nasa-invests-in-visionary-technology/" target="_self">NASA Invests in Visionary Technology </a></div>
[<div class="article_teaser_body">NASA is investing in technology concepts, including several from JPL, that may one day be used for future space exploration missions.</div>, <div class="article_teaser_body">NASA is about to go on a journey to study the center of Mars.</div>, <div class="article_teaser_body">InSight, the next mission to the Red Planet, will use seismology to see into the depths of Mars.</div>, <div class="article_teaser_body">NASA's Mars Curiosity rover just hit a new milestone: its two-thousandth Martian day on the Red Planet. An image mosaic taken recently offers a preview of what comes next.</div>, <div class="article_teaser_body">NASA’s next mission to Mars will be the topic of a media briefing Thursday, March 29, at JPL. The briefing will air live on NASA Television and the agency’s website.</div>, <div class="article

<div class="content_title"><a href="/news/8326/nasa-invests-in-visionary-technology/" target="_self">NASA Invests in Visionary Technology </a></div>
[<div class="article_teaser_body">NASA is investing in technology concepts, including several from JPL, that may one day be used for future space exploration missions.</div>, <div class="article_teaser_body">NASA is about to go on a journey to study the center of Mars.</div>, <div class="article_teaser_body">InSight, the next mission to the Red Planet, will use seismology to see into the depths of Mars.</div>, <div class="article_teaser_body">NASA's Mars Curiosity rover just hit a new milestone: its two-thousandth Martian day on the Red Planet. An image mosaic taken recently offers a preview of what comes next.</div>, <div class="article_teaser_body">NASA’s next mission to Mars will be the topic of a media briefing Thursday, March 29, at JPL. The briefing will air live on NASA Television and the agency’s website.</div>, <div class="article

'NASA Invests in Visionary Technology '